In [44]:
import numpy as np
import pandas as pd
import math
import random
import scipy.stats as stats

### Data

In [45]:
df = pd.read_csv('data.csv') #read data csv

df.drop(columns=['kode_provinsi', 'nama_provinsi', 'bps_kode_kabupaten_kota',
       'bps_kode_kecamatan', 'bps_kode_desa_kelurahan',
       'kemendagri_kode_kecamatan', 'kemendagri_nama_kecamatan',
       'kemendagri_kode_desa_kelurahan', 'kemendagri_nama_desa_kelurahan',
       'tahun', 'id'], inplace=True) #menghapus kolom yang tidak diperlukan

df.rename(columns={'bps_nama_kabupaten_kota':'kabupaten', 'bps_nama_kecamatan':'kecamatan', 'bps_nama_desa_kelurahan':'kelurahan', 'waktu_tempuh_puskes_inap':'waktu_tempuh'}, inplace=True) #mengganti nama kolom untuk mempermudah

In [46]:
# daftar seluruh kecamatan di kab. karawang
list_kecamatan = ['PANGKALAN', 'TELUKJAMBE TIMUR', 'CIAMPEL', 'KLARI',
       'RENGASDENGKLOK', 'KUTAWALUYA', 'BATUJAYA', 'TIRTAJAYA', 'PEDES',
       'CIBUAYA', 'PAKISJAYA', 'CIKAMPEK', 'JATISARI', 'CILAMAYA WETAN',
       'TIRTAMULYA', 'TELAGASARI', 'RAWAMERTA', 'LEMAHABANG', 'TEMPURAN',
       'MAJALAYA', 'JAYAKERTA', 'CILAMAYA KULON', 'BANYUSARI',
       'KOTA BARU', 'KARAWANG TIMUR', 'TELUKJAMBE BARAT', 'TEGALWARU',
       'PURWASARI', 'CILEBAR']

### Pra Survey

In [47]:
# read data pra survey
pra_survey = pd.read_csv('pra_survey.csv')

# buat tabel
pra_survey = pra_survey.groupby('kecamatan').agg({'kelurahan':'count','waktu_tempuh':sum}).rename(columns={'kelurahan':'kelurahan (Mi)','waktu_tempuh':'waktu_tempuh (yi)'})

# tampilkan
pra_survey

,kelurahan (Mi),waktu_tempuh (yi)
kecamatan,,
CIKAMPEK,10,82.0
KUTAWALUYA,12,99.0
MAJALAYA,7,120.0
PEDES,12,391.0
RENGASDENGKLOK,9,180.0
TIRTAMULYA,10,241.0


### Menentukan Ukuran Sampel

In [48]:
# hitung nilai D
N = 29 # jumlah seluruh kecamatan
B = 500 # bound of error

D = (B**2)/(4*(N**2))

print(f"Nilai D adalah sebesar {D:.2f}")

Nilai D adalah sebesar 74.32


In [49]:
# hitung nilai r/mu
Mu = pra_survey['waktu_tempuh (yi)'].sum() / pra_survey['kelurahan (Mi)'].sum()

In [50]:
# kolom pembantu untuk menghitung varians
pra_survey['Mu_Mi'] = pra_survey['kelurahan (Mi)'] * Mu
pra_survey['yi-Mu_Mi'] = pra_survey['waktu_tempuh (yi)'] - pra_survey['Mu_Mi']
pra_survey['yi-Mu_Mi^2'] = pra_survey['yi-Mu_Mi'] ** 2


In [51]:
# hitung varians
varians = pra_survey['yi-Mu_Mi^2'].sum() / (N-1)
varians

print(f"Nilai varians adalah sebesar {varians:.2f}")

Nilai varians adalah sebesar 2060.55


In [52]:
# hitung ukuran sampel
n = (N * varians) / ((N * D) + varians)
n = math.ceil(n)

print(f"Ukuran sampel yang akan diambil adalah sebesar {n} cluster")

Ukuran sampel yang akan diambil adalah sebesar 15 cluster


In [160]:
random.seed(25)
list_sample = random.sample(list_kecamatan, k=n)
list_sample

['JATISARI',
 'KARAWANG TIMUR',
 'TEGALWARU',
 'PANGKALAN',
 'BATUJAYA',
 'CIBUAYA',
 'JAYAKERTA',
 'TELAGASARI',
 'TELUKJAMBE TIMUR',
 'PEDES',
 'BANYUSARI',
 'KOTA BARU',
 'CILAMAYA WETAN',
 'KLARI',
 'LEMAHABANG']

### Penaksiran Parameter

#### Rata-rata

In [161]:
# buat tabel
sample_data = df[df['kecamatan'].isin(list_sample)].groupby('kecamatan').agg({'kelurahan':'count','waktu_tempuh':sum}).rename(columns={'kelurahan':'kelurahan (Mi)','waktu_tempuh':'waktu_tempuh (yi)'})

# tampilkan
sample_data

,kelurahan (Mi),waktu_tempuh (yi)
kecamatan,,
BANYUSARI,12,202.0
BATUJAYA,10,129.0
CIBUAYA,11,182.0
CILAMAYA WETAN,12,108.0
JATISARI,14,160.0
JAYAKERTA,8,75.0
KARAWANG TIMUR,4,15.0
KLARI,13,145.0
KOTA BARU,9,26.0


In [162]:
# hitung sample ratio (r)
r = sample_data['waktu_tempuh (yi)'].sum() / sample_data['kelurahan (Mi)'].sum()

print(f"Total waktu tempuh di kecamatan sampel {sample_data['waktu_tempuh (yi)'].sum():.2f} menit")
print(f"Total kelurahan di kecamatan sampel {sample_data['kelurahan (Mi)'].sum():.0f} kelurahan")
print(f"Rata-rata waktu tempuh adalah {r:.2f} menit")

Total waktu tempuh di kecamatan sampel 2229.00 menit
Total kelurahan di kecamatan sampel 156 kelurahan
Rata-rata waktu tempuh adalah 14.29 menit


#### Varians Cluster Sampel

In [163]:
# kolom pembantu untuk menghitung varians
sample_data['r_Mi'] = sample_data['kelurahan (Mi)'] * r
sample_data['yi-r_Mi'] = sample_data['waktu_tempuh (yi)'] - sample_data['r_Mi']
sample_data['yi-r_Mi^2'] = sample_data['yi-r_Mi'] ** 2

In [164]:
# hitung varians cluster sampel
sc_sqr = sample_data['yi-r_Mi^2'].sum()/(n-1)

print(f"Varians cluster sampel adalah ({np.sqrt(sc_sqr):.2f})^2")

Varians cluster sampel adalah (89.45)^2


#### Varians dari Total Populasi

In [165]:
# hitung varians dari total populasi
var_est = N*(N-n)*sc_sqr/n

print(f"Varians dari total populasi adalah ({np.sqrt(var_est):.2f})^2")

Varians dari total populasi adalah (465.38)^2


#### Varians dari Rata-Rata

In [166]:
# hitung M
M_bar = np.mean(sample_data['kelurahan (Mi)'])

# hitung varians dari rata-rata
var_mean = var_est / (N*M_bar)**2

print(f"Varians dari rata-rata adalah ({np.sqrt(var_mean):.2f})^2")

Varians dari rata-rata adalah (1.54)^2


#### 95% Confidence Interval

In [171]:
# hitung z stat dengan 95% CI
alpha = 0.05
z_stat = stats.norm.ppf(1-alpha/2)

# hitung nilai d
d = z_stat * np.sqrt(var_mean)

# hitung 95% CI
lower_range = r - d
upper_range = r + d

print(f"Dengan 95% CI, rata-rata waktu tempuh ke Puskesmas berada di {lower_range:.2f} s.d {upper_range:.2f} menit")

Dengan 95% CI, rata-rata waktu tempuh ke Puskesmas berada di 11.26 s.d 17.31 menit
